# Getting started with `sbi`

Note, you can find the original version of this notebook at [https://github.com/sbi-dev/sbi/blob/main/tutorials/00_getting_started.ipynb](https://github.com/sbi-dev/sbi/blob/main/tutorials/00_getting_started.ipynb) in the `sbi` repository.

In [1]:
import torch
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer

In [2]:
from __future__ import annotations

import pytest
import torch
from torch import eye, ones, zeros
from torch.distributions import MultivariateNormal

from sbi import utils
from sbi.inference import SNLE, SNPE, SNRE
from sbi.neural_nets.embedding_nets import (
    CNNEmbedding,
    FCEmbedding,
    PermutationInvariantEmbedding,
)
from sbi.simulators.linear_gaussian import (
    linear_gaussian,
    true_posterior_linear_gaussian_mvn_prior,
)
from sbi.utils import classifier_nn, likelihood_nn, posterior_nn

import pytest
from torch import eye, ones, zeros
from torch.distributions import MultivariateNormal

from sbi.inference import (
    SNPE_A,
    SNPE_C,
    DirectPosterior,
    prepare_for_sbi,
    simulate_for_sbi,
)
from sbi.simulators.linear_gaussian import diagonal_linear_gaussian

from sbi.utils.metrics import c2st

from sbi.inference import (
    SNLE,
    SNPE,
    DirectPosterior,
    MCMCPosterior,
    likelihood_estimator_based_potential,
    prepare_for_sbi,
    simulate_for_sbi,
)

In [3]:
from torch.distributions import MultivariateNormal, Uniform
num_dim = 1
x_o = torch.tensor([[1.0]])
num_samples = 100

# likelihood_mean will be likelihood_shift+theta
likelihood_shift = -1.0 * ones(num_dim)
likelihood_cov = 0.3 * eye(num_dim)

prior = Uniform(low=torch.zeros(num_dim), high=torch.ones(num_dim))

def simulator(theta: Tensor) -> Tensor:
    return linear_gaussian(theta, likelihood_shift, likelihood_cov)

simulator, prior = prepare_for_sbi(simulator, prior)
inference = SNPE(density_estimator="mdn")

theta, x = simulate_for_sbi(simulator, prior, 100)
posterior_estimator = inference.append_simulations(theta, x).train()
posterior = DirectPosterior(posterior_estimator=posterior_estimator, prior=prior)
samples = posterior.sample((num_samples,), x=x_o)
log_probs = posterior.log_prob(samples, x=x_o)

assert log_probs.shape == torch.Size([num_samples])

/root/sbi/sbi/utils/user_input_checks.py:209: UserWarning: Casting 1D Uniform prior to BoxUniform to match sbi batch requirements.
  warnings.warn(


Running 100 simulations.:   0%|          | 0/100 [00:00<?, ?it/s]

/root/miniconda3/envs/sbi3/lib/python3.11/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


 Neural network successfully converged after 122 epochs.

Drawing 100 posterior samples:   0%|          | 0/100 [00:00<?, ?it/s]

torch.Size([1, 100, 1])
torch.Size([1, 100])
torch.Size([68, 1])
torch.Size([1, 100])
torch.Size([1, 100, 1])
torch.Size([1, 100])
torch.Size([73, 1])
torch.Size([1, 100])
torch.Size([1, 10000, 1])
torch.Size([1, 10000])
torch.Size([7108, 1])
torch.Size([1, 10000])
torch.Size([1, 6102, 1])
torch.Size([1, 6102])
torch.Size([4313, 1])
torch.Size([1, 6102])


In [4]:
samples.shape

torch.Size([100, 1])

In [7]:
samples

tensor([], size=(100, 0))

In [6]:
x_o

tensor([[1.]])

In [16]:
posterior.posterior_estimator.sample((10,), condition=xo).shape

torch.Size([1, 10, 32])

In [18]:
xo.shape

torch.Size([1, 32])

In [4]:
posterior.sample((1,), x=x_o).shape

Drawing 1 posterior samples:   0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([1, 2])

In [15]:
x_o.shape

torch.Size([1, 2])

## Running the inference procedure

`sbi` provides a simple interface to run state-of-the-art algorithms for simulation-based inference.

For inference, you need to provide two ingredients:

1) a prior distribution that allows to sample parameter sets.  
2) a simulator that takes parameter sets and produces simulation outputs.

For example, we can have a 3-dimensional parameter space with a uniform prior between [-1,1] and a simple simulator that for the sake of example adds 1.0 and some Gaussian noise to the parameter set:

In [3]:
num_dim = 3
prior = utils.BoxUniform(low=-2 * torch.ones(num_dim), high=2 * torch.ones(num_dim))

def simulator(parameter_set):
    return 1.0 + parameter_set + torch.randn(parameter_set.shape) * 0.1

`sbi` can then run inference:

In [4]:
# Other methods are "SNLE" or "SNRE".
posterior = infer(simulator, prior, method="SNPE", num_simulations=1000)

Running 1000 simulations.:   0%|          | 0/1000 [00:00<?, ?it/s]

 Neural network successfully converged after 67 epochs.

Let's say we have made some observation $x$:

In [6]:
observation = torch.zeros(3)

 Given this observation, we can then sample from the posterior $p(\theta|x)$, evaluate its log-probability, or plot it.

In [7]:
samples = posterior.sample((10000,), x=observation)
log_probability = posterior.log_prob(samples, x=observation)
_ = analysis.pairplot(samples, limits=[[-2, 2], [-2, 2], [-2, 2]], figsize=(6, 6))

Drawing 10000 posterior samples:   0%|          | 0/10000 [00:00<?, ?it/s]

                    accepted. It may take a long time to collect the remaining
                    9999 samples. Consider interrupting (Ctrl-C) and switching to
                    `build_posterior(..., sample_with='mcmc')`.


KeyboardInterrupt: 

## Next steps

The single-line interface described above provides an easy entry for using `sbi`. However, on almost any real-world problem that goes beyond a simple demonstration, we strongly recommend using the [flexible interface](https://sbi-dev.github.io/sbi/tutorial/02_flexible_interface/).